In [1]:
import pip

!pip install Pillow

In [2]:
"""
pip install opencv-contrib-python==3.4.2.16
"""
import base64
import json
import pprint
import cv2
import numpy as np
from finalProject.classes.yolo import Yolo
from finalProject.utils.drawing.draw import draw_final_results
from finalProject.utils.images.imagesUtils import Image
from finalProject.utils.keyPoints.AlgoritamKeyPoints import create_keypoints_descriptors, sift_keypoints_detection
from finalProject.utils.matchers.Matchers import compare_between_descriptors, flann_matcher
from finalProject.utils.preprocessing.preprocess import read_frames_from_video, check_frames_exist, reduce_noise, removeRemovalColor
from finalProject.utils.tracking.TrackingByYolo import source_detection_by_yolo, tracking_by_yolo
from finalProject.classes.results_details import ResultsDetails
import matplotlib.pyplot as plt
import random
import pandas as pd
import io
from PIL import Image
import os
import datetime

def sobel(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    sobel_intensity = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx, 2.0), 1.0, cv2.pow(sobely, 2.0), 1.0, 0.0))
    return sobel_intensity


def sobel_keypoints(image):
    sobelImage = sobel(image)
    # norm
    image8bit = cv2.normalize(sobelImage, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')

    h = image8bit.shape[0]
    w = image8bit.shape[1]
    three = h // 3  # the floor division // rounds the result down to the nearest whole number

    k_lower, d_lower = sift_keypoints_detection(image8bit[0:three, :])
    k_middle, d_middle = sift_keypoints_detection(image8bit[three:2 * three, :])
    k_higher, d_higher = sift_keypoints_detection(image8bit[2 * three:, :])

    return (k_lower, k_middle, k_higher), (d_lower, d_middle, d_higher), image8bit


def forward(frames):
    outputs = []
    for frame in frames:
        frame.gray_image = sobel(frame.gray_image)
        key_des_image = sobel_keypoints(frame.gray_image)
        outputs.append((key_des_image, frame.frame_image))
    return outputs


def saveImage(image, folder_name, counter):
    image_name = "image_" + str(counter)
    path = "./reports/" + str(folder_name) + "/" + image_name + ".png"
    cv2.imwrite(path, image)
    return image_name


def convert(folder_name):
    def convertImagetoIds(output, counter):
        output[0] = saveImage(output[0], folder_name, "comparation_" + counter)
        output[1] = saveImage(output[1], folder_name, "source_" + counter)
        output[2] = saveImage(output[2], folder_name, "target_" + counter)
        return output[3:]
    return convertImagetoIds


def exportToExcel(outputs, input_data):
    curr_time = datetime.datetime.now()
    folder_name = "folderReport_" + str(curr_time.strftime("%d-%m-%Y_%I%p-%Mm"))
    path = "./reports/" + folder_name
    print("creating report on: ", path)
    os.makedirs(path, mode=0o777, exist_ok=False)

    print("number of outputs ", len(outputs))
    if len(outputs) == 0:
        print("no found match")
        return None

    counts = []
    for i, v in enumerate(outputs):
        counts.append(str(i))

    excel_table = list(map(convert(folder_name), outputs, counts))
    df1 = pd.DataFrame(np.array(excel_table),
                       columns=['len(source_keypoints)', 'len(target_keypoints)',
                                'len(total_matches)', 'accuracy'])

    details_table = []
    details_table.append([input_data[0].input_name.split("/")[-1],
                          input_data[0].frame_rate, input_data[0].reduce_noise, input_data[0].num_of_people,
                          input_data[0].num_of_frames])
    details_table.append([input_data[1].input_name.split("/")[-1],
                          input_data[1].frame_rate, input_data[1].reduce_noise, input_data[1].num_of_people,
                          input_data[1].num_of_frames])

    df2 = pd.DataFrame(np.array(details_table),
                       index=['source', 'target'],
                       columns=['inputVideo', 'frame_rate',
                                'reduceNoise', 'num_of_people', 'num_of_frames'])

    with pd.ExcelWriter("./reports/" + folder_name + "/report_" + folder_name + ".xls") as writer:
        df1.to_excel(writer, float_format="%.3f", sheet_name='accuracy')
        df2.to_excel(writer, sheet_name='details')


def sliceImageToThreeParts(keys, descriptors, image):
    h, w, channels = image.shape

    upperY = 0
    middleHigherY = h // 3
    middleLowerY = (h // 3) * 2
    lowerY = h

    # start split keys    
    upperKeys = list(filter(lambda key: upperY <= key.pt[1] <= middleHigherY, keys))


def cross_correction(key_des_image_source, key_des_image_target, threshold=0.8):
    ((k1s, k2s, k3s), (d1s, d2s, d3s), source_img) = key_des_image_source[0]
    ((k1t, k2t, k3t), (d1t, d2t, d3t), target_img) = key_des_image_target[0]

    match1 = flann_matcher(d1s, d1t, threshold=threshold)
    match2 = flann_matcher(d2s, d2t, threshold=threshold)
    match3 = flann_matcher(d3s, d3t, threshold=threshold)

    src_h = key_des_image_source[1].shape[0]
    src_w = key_des_image_source[1].shape[1]
    src_three = src_h // 3  # the floor division // rounds the result down to the nearest whole number

    trg_h = key_des_image_target[1].shape[0]
    trg_w = key_des_image_target[1].shape[1]
    trg_three = trg_h // 3  # the floor division // rounds the result down to the nearest whole number

    output = np.zeros((max(src_h, trg_h), src_w + trg_w, 3), np.uint8)

    output[0:src_three, :] = cv2.drawMatchesKnn(key_des_image_source[1][0:src_three, :], k1s,
                                                key_des_image_target[1][0:src_three, :], k1t, match1, outImg=None)
    output[src_three:src_three * 2, :] = cv2.drawMatchesKnn(key_des_image_source[1][src_three:src_three * 2, :], k2s,
                                                            key_des_image_target[1][src_three:src_three * 2, :], k2t,
                                                            match2, outImg=None)
    output[src_three * 2:, :] = cv2.drawMatchesKnn(key_des_image_source[1][src_three * 2:, :], k3s,
                                                   key_des_image_target[1][src_three * 2:, :], k3t, match3, outImg=None)

    match = match1 + match2 + match3

    if len(match) == 0:
        return None

    keys_s = (len(k1s) + len(k2s) + len(k3s))
    keys_t = (len(k1s) + len(k2s) + len(k3s))
    acc = len(match) / keys_s

    return [output, key_des_image_source[1], key_des_image_target[1], keys_s, keys_t, len(match),
            min(acc, 1)]  # item = (outputs,source_img,target_img,len(k1),len(k2),acc..)


def sort_by_acc_func(output_row):
    return output_row[6]


def plotSquare(outputs, _titles, threshold, each_column=10):
    images = list(map(lambda output: output[0], outputs))
    number_images = len(images)
    rows = (number_images // each_column)
    index = 1

    print(rows)
    fig, axes = plt.subplots(rows, each_column, figsize=(48, 24), dpi=300)
    fig.subplots_adjust(hspace=0.3, wspace=0.05)

    for ax, image, title in zip(axes.flat, images, _titles):
        ax.imshow(image)
        ax.set_title(title)
        index += 1

    rand = random.uniform(0, 1)
    pathX = "./figures/fig_test" + str(rand) + "_thresh_" + str(threshold) + ".svg"
    print("saving on", pathX)
    plt.savefig(pathX, dpi=fig.dpi)
    plt.clf()
    plt.close()


In [3]:
# init yolo
yolo = Yolo()
yolo.initYolo()
pp = pprint.PrettyPrinter(indent=4)


[INFO] loading YOLO from disk...


In [4]:
def run(threshold=0.8):
  
    with open('./config.txt') as file_json:
        config = json.load(file_json)

        # source
        source_frames = read_frames_from_video(config["source"])
        if not check_frames_exist(source_frames):
            print("problem with the input of source video.")
            exit(0)

        source_person = source_detection_by_yolo(source_frames, yolo,
                                                 is_video=config["source"]["isVideo"],
                                                 config=config["source"])
        if source_person is None:
            print("failed to detect a person on source video")
            exit(0)

        # target
        target_frames = read_frames_from_video(config["target"])
        if not check_frames_exist(target_frames):
            print("problem with the input of target video.")
            exit(0)

        target_people = tracking_by_yolo(target_frames, yolo, is_video=config["target"]["isVideo"], config=config["target"])

        cv2.destroyAllWindows()
        
        # source
        sources_output = forward(source_person.frames)  # array of outputs (key,des,image)
        
        targets_outputs = []  # array of targets , each target have array of frames
        for target_person in target_people:
            target_output = forward(target_person.frames)  # array of frames
            targets_outputs.append(target_output)

        outputs = []  # each frame

        for source_frame in sources_output:
            for target in targets_outputs:
                for target_frame in target:  # threshold for match ! bigger - more lines
                    cross = cross_correction(source_frame, target_frame, threshold)
                    if cross is not None:
                        outputs.append(cross)

        outputs.sort(reverse=True, key=sort_by_acc_func)
        
        src_input_data = ResultsDetails(config["source"])
        trg_input_data = ResultsDetails(config["target"])
        
        src_input_data.num_of_frames = len(source_person.frames)
        print("src_input_data.num_of_frames: ", src_input_data.num_of_frames)

        trg_input_data.num_of_people = len(target_people)
        print("number of people: ", trg_input_data.num_of_people)
        trg_num_of_frames = 0
        for p in target_people:
            trg_num_of_frames += len(p.frames)
        trg_input_data.num_of_frames = trg_num_of_frames
        print("trg_num_of_frames: ", trg_input_data.num_of_frames)
        input_data = (src_input_data, trg_input_data)
        exportToExcel(outputs[:10], input_data)
                        
        titles = ["test"] * (len(outputs) - 1)
        plotSquare(outputs, titles,threshold)
        print("finish")

In [5]:
for thresh in [0.8]:
    run(threshold=thresh)


src_input_data.num_of_frames:  20
number of people:  1
trg_num_of_frames:  20
creating report on:  ./reports/folderReport_03-07-2020_11PM-59m
number of outputs  10
finish
